In [230]:
from bs4 import BeautifulSoup as bs
from datetime import datetime as dt, timedelta
import requests
import re

In [231]:
current_date = dt.now().date()
current_time = dt.now().time().hour
days_to_capture = 4
year = current_date.year

url = 'https://www.bbc.co.uk/weather/2643743'
head = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'en-GB,en;q=0.5',
    'Connection': 'keep-alive',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:128.0) Gecko/20100101 Firefox/128.0'
}

forecast_class = 'wr-time-slot__inner'
hours_class = 'wr-time-slot-primary__hours'
description_class = 'wr-time-slot-primary__weather-type-description'
temperature_class = 'wr-value--temperature--c'
precipitation_class = 'wr-time-slot-primary__precipitation'
rain_text_class = 'wr-u-font-weight-500'

In [232]:
weather_data = []

for i in range(days_to_capture):
    date = current_date + timedelta(days=i)
    response = requests.get(url + '/day' + str(i), headers=head)
    
    if i == 0:
        hours = ['12', '13', '15', '18', '00']
    else:
        hours = ['12']

    if response.status_code == 200:
        soup = bs(response.text, 'html.parser')
        forecast = soup.select(f'button[class^="{forecast_class}"]')

        if forecast:
            #weather_data.append(return_forecast(forecast, date, hours))
            for item in forecast:
                hour = item.find('span', class_=f'{hours_class}')
                type = None
                temp = None
                chance = None

                if hour.text in hours:
                    type = item.find('div', class_=f'{description_class}')
                    
                    temp = item.find('span', class_=f'{temperature_class}')
                    if temp:
                        temp = re.search(r'\d+', temp.text)
                        if temp:
                            temp = temp.group()

                    rain = item.find('div', class_=f'{precipitation_class}')
                    if rain:
                        chance = rain.find('div', class_=f'{rain_text_class}')
                        if chance:
                            chance = re.search(r'\d+', chance.text)
                            if chance:
                                chance = chance.group()

                    weather_data.append([date, 1, i, int(hour.text), type.text.lower(), int(temp), int(chance)])

print(weather_data)

[[datetime.date(2024, 8, 9), 1, 0, 18, 'sunny', 23, 0], [datetime.date(2024, 8, 9), 1, 0, 0, 'clear sky', 17, 0], [datetime.date(2024, 8, 10), 1, 1, 12, 'light cloud', 21, 0], [datetime.date(2024, 8, 11), 1, 2, 12, 'sunny intervals', 25, 0], [datetime.date(2024, 8, 12), 1, 3, 12, 'sunny', 29, 0]]


In [233]:
weather_data = []

url = 'https://www.yr.no/en/forecast/daily-table/2-2643743/United%20Kingdom/England/Greater%20London/London'
head = {
    'Accept': 'image/avif,image/webp,image/png,image/svg+xml,image/*;q=0.8,*/*;q=0.5',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'en-GB,en;q=0.5',
    'Connection': 'keep-alive',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:128.0) Gecko/20100101 Firefox/128.0'
}

current_class = 'now-hero__next-hour-content'
response = requests.get(url, headers=head)

if response.status_code == 200:
    soup = bs(response.text, 'html.parser')
    current_weather = soup.select('div[class^="now-hero__next-hour-content"]')

    if current_weather:
        for item in current_weather:
            type = item.find('img', class_='weather-symbol__img')
            if type:
                type = type.get('alt')

            temp = item.find('span', class_=re.compile(r'^temperature'))
            if temp:
                temp = re.search(r'\d+', temp.text)
                if temp:
                    temp = temp.group()

            rain = item.find('span', 'now-hero__next-hour-precipitation-value')
            if rain:
                rain = rain.text

            weather_data.append([current_date, 0, 0, current_time, type, int(temp), int(rain)])

            break

print(weather_data)

[[datetime.date(2024, 8, 9), 0, 0, 17, 'partly cloudy', 24, 0]]
